# 🔌 4 Function Calling

⚠️ Note function-calling compatible GPT models: (https://platform.openai.com/docs/guides/function-calling).

### Initialize the kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"

#!import Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Settings.InitializeKernel();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha

### ⌚️ Let's use a native function written in C#.

In [2]:
using System.ComponentModel;
public class DateTimePlugin
{
    [KernelFunction]
    [Description("Retrieves the current date.")]
    public string GetCurrentDate() => DateTime.Now.ToShortDateString();
}

kernel.ImportPluginFromType<DateTimePlugin>("date");

### Let's prepare the execution settings and the kernel arguments.

In [3]:
var executionSettings = new OpenAIPromptExecutionSettings
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions 
};

var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "Christmas"
};

var promptTemplate = "How many days until {{$input}}?";

var response = await kernel.InvokePromptAsync(promptTemplate, kernelArguments);

display(response.GetValue<string>())

Today is April 18, 2024. Since Christmas is on December 25, the number of days until Christmas from today's date is:

- December has 31 days, so from December 1 to December 25, there are 25 days.
- The number of days remaining in April after the 18th is 30 - 18 = 12 days.
- The months of May, June, July, August, September, October, and November have a total of:
  - May: 31 days
  - June: 30 days
  - July: 31 days
  - August: 31 days
  - September: 30 days
  - October: 31 days
  - November: 30 days

Let's add them all up:

12 days (remaining in April) + 31 (May) + 30 (June) + 31 (July) + 31 (August) + 30 (September) + 31 (October) + 30 (November) + 25 (December) =

12 + 31 + 30 + 31 + 31 + 30 + 31 + 30 + 25 = 251 days.

There are 251 days until Christmas from April 18, 2024.

### 🏃💨 This is the same example as above, but with streaming too

In [4]:
var executionSettings = new OpenAIPromptExecutionSettings
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions 
};

var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "Christmas"
};

await foreach (var update in kernel.InvokePromptStreamingAsync("How many days until {{$input}}?", kernelArguments))
{
    Console.Write(update);
}

Today is April 18, 2024. To calculate the number of days until Christmas:

Christmas is on December 25, 2024.

From April 18 to May 18 is 30 days.
From May 18 to December 18 is 7 full months. Since the months May, July, August, October, and December have 31 days, and June and September have 30 days, that's:
4 months with 31 days = 4 * 31 = 124 days
2 months with 30 days = 2 * 30 = 60 days
1 month with 31 days (July) since we don't count December itself just yet.
So, 124 days + 60 days + 31 days = 215 days
Then, from December 18 to December 25 is another 7 days.

Adding those intervals together gives us:
30 days (April 18 to May 18) + 215 days (May 18 to December 18) + 7 days (December 18 to December 25) = 30 + 215 + 7 = 252 days.

There are 252 days until Christmas from April 18, 2024.

# Manual

In [6]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text.Json;
using System.Threading.Tasks;
using Azure.AI.OpenAI;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var chat = kernel.GetRequiredService<IChatCompletionService>();
var chatHistory = new ChatHistory();

var executionSettings = new OpenAIPromptExecutionSettings
{
    ToolCallBehavior = ToolCallBehavior.EnableKernelFunctions 
};

chatHistory.AddUserMessage("How many days until Christmas?");
while (true)
{
    var result = (OpenAIChatMessageContent)await chat.GetChatMessageContentAsync(chatHistory, executionSettings, kernel);

    if (result.Content is not null)
    {
        Console.Write(result.Content);
    }

    List<ChatCompletionsFunctionToolCall> toolCalls = result.ToolCalls.OfType<ChatCompletionsFunctionToolCall>().ToList();
    if (toolCalls.Count == 0)
    {
        break;
    }

    chatHistory.Add(result);
    foreach (var toolCall in toolCalls)
    {
        string content = kernel.Plugins.TryGetFunctionAndArguments(toolCall, out KernelFunction? function, out KernelArguments arguments) ?
            JsonSerializer.Serialize((await function.InvokeAsync(kernel, arguments)).GetValue<object>()) :
            "Unable to find function. Please try again!";

        if (function != null)
        {
            Console.WriteLine($"  >> 🔌 {toolCall.Name.ToString()}: /{function.Name}");
        }

        Console.WriteLine($"       Result: {content}");

        chatHistory.Add(new ChatMessageContent(
            AuthorRole.Tool,
            content,
            metadata: new Dictionary<string, object>(1) { { OpenAIChatMessageContent.ToolIdProperty, toolCall.Id } }));
    }
}

  >> 🔌 date-GetCurrentDate: /GetCurrentDate
       Result: "4/18/2024"
Today is April 18, 2024. Christmas Day is on December 25th. To calculate how many days until Christmas:

From April 18 to April 30 is 12 days (completing April).
From May to November is 7 full months. Given that May, July, August, October, and December have 31 days, and June and September have 30 days, and November has 30 days, the total is: 31 (May) + 30 (June) + 31 (July) + 31 (August) + 30 (September) + 31 (October) + 30 (November) = 214 days.
From December 1 to December 25 is 24 days.

Adding these up: 12 + 214 + 24 = 250 days until Christmas Day on December 25, 2024.